In [ ]:
# default_exp metrics

# Metrics

> This contains metrics not included in fastai.

In [ ]:
#export
from tsai.imports import *
from fastai.metrics import * 

In [ ]:
#export
mk_class('ActivationType', **{o:o.lower() for o in ['No', 'Sigmoid', 'Softmax', 'BinarySoftmax']},
         doc="All possible activation classes for `AccumMetric")

In [ ]:
#export
def MatthewsCorrCoefBinary(thresh=.5, sample_weight=None):
    "Matthews correlation coefficient for single-label classification problems"
    return skm_to_fastai(skm.matthews_corrcoef, activation=ActivationType.BinarySoftmax, thresh=thresh, sample_weight=sample_weight)

In [ ]:
#export
def get_task_metrics(dls, binary_metrics=None, multi_class_metrics=None, regression_metrics=None, verbose=True): 
    if dls.c == 2: 
        pv('binary-classification task', verbose)
        return binary_metrics
    elif dls.c > 2: 
        pv('multi-class task', verbose)
        return multi_class_metrics
    else:
        pv('regression task', verbose)
        return regression_metrics

In [ ]:
#hide
out = create_scripts(); beep(out)